<a href="https://colab.research.google.com/github/nomi181472/MS-DS-Practice/blob/main/MS_DS_Assignment_1_17k8069.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing packages**

In [ ]:
import pandas as pd
import numpy as np

**Utilities**

In [ ]:

def calculate_imputing_values(col,cdf):
  ser=cdf[col]
  if ser.dtype=='int64':
    return round(cdf[col].mean(),2)
  elif ser.dtype=='float64':
    return  round(cdf[col].mean(),2)
  elif ser.dtype=='object':
    return cdf[col].value_counts(ascending=True).index[0]
  assert False,"please add your custom datatype in calculate_imputing_values"
def handle_missing_values(row,cdf):
  na_rows=row.isna()
  if na_rows.sum()==0:# row is ok
    return row
  else: # row contains missing values, need to be handle
    is_need_to_remove=na_rows.sum()>int(len(row)/threshold)
    if is_need_to_remove: # row need to be removed.
      row[:] = np.nan
      return row
    else:  #impute the value on missing cells in a row
      row_which_need_to_filled=row.loc[row.isna()]
      for ind in row_which_need_to_filled.index:
        imputing_value=calculate_imputing_values(ind,cdf)
        row.loc[ind]=imputing_value
  return row
def convert_dtype(col):
  column_name=col.name
  if actual_types[column_name]==col.dtype:
    return col
  else:
    if actual_types[column_name]=="datetime64":
      return pd.to_datetime(col)
    else:
      return col.astype(actual_types[column_name])
def check_dtypes(cdf):
  return {col: "OK" if cdf[col].dtype==actual_types[col] else "Required changed"  for col in cdf}
def calculate_zscore(cdf,col):
  zscore=(cdf[col]-cdf[col].mean())/cdf[col].std()
  return zscore

def check_outlier_threshold(zscores,out):
  return  ~(zscores < out[0]) & (zscores > out[-1])

def handle_gender(datum):
  datum=datum.lower()
  if datum in gender_categorical_values:
    return gender_categorical_values[datum].upper()
  return datum.upper()
def remove_white_spaces(col):
  if col.dtype=="string":
    return col.str.strip()
  return col
def validate_email(cell):
  return all([s in cell for s in ["@","."]])
def winsorizing(cell):
  if cell>gpa_threshold:
    return gpa_threshold
  elif cell<-1:
    return gpa_min_threshold
  return cell
def email_winsorizing(cell):
  if validate_email(cell):
    return cell
  index=cell.find("email") # as there is common email word
  if index==-1:
    return None
  cell=cell[:index]+"@"+cell[index:]
  return cell

**Configurations And Settings**

In [ ]:
threshold=2
outliers_threshold =(-1.5,1.5)
gpa_threshold=4
gpa_min_threshold=1
gender_categorical_values={"male":"M","female":"F","M":"M","F":"F"}
actual_types={"StudentID":"int64","Name":"string","Age":"int32","Gender":"category","Email":"string","Phone":"string","City":"string","GPA":"float32","Join_Date":"datetime64"}
# Creating a sample dataset with data quality issues
csv_file_path='dirty_data.csv'
data = {
    'StudentID': [1, 2, 3, 4, 5],
    'Name': ['John Doe', 'Jane Smith', 'Robert Johnson', 'Emily Davis', 'Michael Brown'],
    'Age': [25, None, 22, 21, 26],
    'Gender': ['Male', 'F', 'M', 'Female', 'Male'],
    'Email': ['johndoeemail.com', 'janesmith@email.com', None, 'emily@email.com', 'michael@email.com'],
    'Phone': ['123-456-7890', '987-654-3210', '555-555-5555', '111-222-3333', '999-888-7777'],
    'City': ['New York', 'Los Angeles', 'Chicago', 'San Francisco', 'Houston'],
    'GPA': [3.5, 4.1, 5.5, 5.0, 3.8],
    'Join_Date': ['2020-01-15', '2019-12-10', '2021-02-18', '2020/03/25', '2018-11-05']
}

**Generate Dataset**

In [ ]:




# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)
df


,StudentID,Name,Age,Gender,Email,Phone,City,GPA,Join_Date
0,1,John Doe,25.0,Male,johndoeemail.com,123-456-7890,New York,3.5,2020-01-15
1,2,Jane Smith,NaN,F,janesmith@email.com,987-654-3210,Los Angeles,4.1,2019-12-10
2,3,Robert Johnson,22.0,M,None,555-555-5555,Chicago,5.5,2021-02-18
3,4,Emily Davis,21.0,Female,emily@email.com,111-222-3333,San Francisco,5.0,2020/03/25
4,5,Michael Brown,26.0,Male,michael@email.com,999-888-7777,Houston,3.8,2018-11-05


**1. Load the Dataset**

In [ ]:
df=pd.read_csv(csv_file_path)
df

,StudentID,Name,Age,Gender,Email,Phone,City,GPA,Join_Date
0,1,John Doe,25.0,Male,johndoeemail.com,123-456-7890,New York,3.5,2020-01-15
1,2,Jane Smith,NaN,F,janesmith@email.com,987-654-3210,Los Angeles,4.1,2019-12-10
2,3,Robert Johnson,22.0,M,NaN,555-555-5555,Chicago,5.5,2021-02-18
3,4,Emily Davis,21.0,Female,emily@email.com,111-222-3333,San Francisco,5.0,2020/03/25
4,5,Michael Brown,26.0,Male,michael@email.com,999-888-7777,Houston,3.8,2018-11-05


**2. Data Inspection**

In [ ]:
#Load the Dataset:

print(f'The number of rows and columns in the dataset: {df.shape}',end="\n\n")
print(f"Data types of each columns.\n{df.dtypes}",end="\n\n")
nan_series=df.isna().sum()
print(f"counting of missing values of each column.\n{nan_series}",end="\n\n")
#we can check rows duplication only through StudentID column, as this column's name suggest that id will always be unique.
print(f"The number of duplicate records: {df.duplicated(subset=['StudentID']).sum()}",end="\n\n")


The number of rows and columns in the dataset: (5, 9)

Data types of each columns.
StudentID      int64
Name          object
Age          float64
Gender        object
Email         object
Phone         object
City          object
GPA          float64
Join_Date     object
dtype: object

counting of missing values of each column.
StudentID    0
Name         0
Age          1
Gender       0
Email        1
Phone        0
City         0
GPA          0
Join_Date    0
dtype: int64

The number of duplicate records: 0



**3. Handling Missing Values**

In [ ]:
nan_series=df.isna().sum()
nan_colum_series=nan_series[nan_series>0]
print(f"columns with missing values:\n{nan_colum_series}",end="\n\n")
#There are multiple ways for handling missing values.
#1 fill by forward values
ffilled_df=df.fillna(method="ffill")
#2 fill by backward values
bfilled_df=df.fillna(method="bfill")
#3 fill float or integer based type columns  by  measures of central tendency (mean, median, mode)
mean=df['Age'].mean()
df['Age'].fillna(mean)
#4 remove rows or columns in which values are missing
droped_df=df.dropna(subset=nan_colum_series.index)
#5 implement some hybrid approach (filling or removing) on each row.
# threshold=2
# Remove values where no of missing values in a row > int(columns.length/threshold)  e.g (NaN,NaN,NaN,value,value) 3 missing values > int(5/threshold) 2
# Fill values where no of miss values in a row <= int(columns.length/threshold)  e.g (NaN,NaN,value,value,value)  2 missing values <= int(5/threshold) 2
handled_missing_df=df.apply(handle_missing_values,args=(df,),axis=1).dropna(axis=0).reset_index(drop=True)
nan_series=handled_missing_df.isna().sum()
nan_colum_series=nan_series[nan_series>0]
print(f"df after handling missig values.\n{nan_colum_series}",end="\n\n")
print(f"new df")
handled_missing_df

#{col:pre_imputing_data(col,df) for col in df.columns}


columns with missing values:
Age      1
Email    1
dtype: int64

df after handling missig values.
Series([], dtype: int64)

new df


,StudentID,Name,Age,Gender,Email,Phone,City,GPA,Join_Date
0,1,John Doe,25.0,Male,johndoeemail.com,123-456-7890,New York,3.5,2020-01-15
1,2,Jane Smith,23.5,F,janesmith@email.com,987-654-3210,Los Angeles,4.1,2019-12-10
2,3,Robert Johnson,22.0,M,johndoeemail.com,555-555-5555,Chicago,5.5,2021-02-18
3,4,Emily Davis,21.0,Female,emily@email.com,111-222-3333,San Francisco,5.0,2020/03/25
4,5,Michael Brown,26.0,Male,michael@email.com,999-888-7777,Houston,3.8,2018-11-05


**4. Handling Duplicate Records**

In [ ]:
subset=["StudentID"]
duplicated=handled_missing_df.duplicated(keep="last",subset=subset)
handled_duplicate_value=handled_missing_df.copy()
if duplicated.sum()>0:
  print(f"duplicate records occured.\n{handled_duplicate_value[duplicated][subset]}")
  handled_duplicate_value.drop_duplicates(subset=subset,inplace=True)
  handled_duplicate_value.reset_index(inplace=True,drop=True)
else:
  print(f"found no duplicated records based on columns: {subset}")


found no duplicated records based on columns: ['StudentID']


**5. Data Type Conversion**

In [ ]:

changes=check_dtypes(handled_duplicate_value)
report=f"Checking which columns need to be check:{changes}"
print(report,end="\n\n")
handled_data_types=handled_duplicate_value.apply(convert_dtype,axis=0,)
print(f"After data type conversion.\n{handled_data_types.dtypes}")
print(handled_data_types.shape)



Checking which columns need to be check:{'StudentID': 'OK', 'Name': 'Required changed', 'Age': 'Required changed', 'Gender': 'Required changed', 'Email': 'Required changed', 'Phone': 'Required changed', 'City': 'Required changed', 'GPA': 'Required changed', 'Join_Date': 'Required changed'}

After data type conversion.
StudentID             int64
Name                 string
Age                   int32
Gender             category
Email                string
Phone                string
City                 string
GPA                 float32
Join_Date    datetime64[ns]
dtype: object
(5, 9)


**6. Outlier Detection and Handling**

In [ ]:
#Identify columns with potential outliers.
#we can detect outliers on the following columns Age, Gender, City, GPA
handled_outliers=handled_data_types.copy()
print(handled_outliers.shape)
numerical_columns=["Age", ]
#for numerical dtypes zscore  and  text based dtypes frequency-based outlier detection are used
outliers={f"{col}":handled_outliers[check_outlier_threshold(calculate_zscore(handled_outliers,col),outliers_threshold)][col].values.tolist() for col in numerical_columns}
print(f"checking outliers between {outliers_threshold} using zscore:\n {outliers}")
#we can remove outliers or impute values
for key in outliers:
  handled_outliers=(handled_outliers[~handled_outliers[key].isin(outliers[key])])
print("checking GPA column outliers")
print(handled_outliers[handled_outliers[ "GPA"]>gpa_threshold]["GPA"].values.tolist())
#handled_outliers=handled_outliers[handled_outliers[ "GPA"]<=gpa_threshold]

handled_outliers["GPA"]=handled_outliers["GPA"].apply(winsorizing)
handled_outliers["Gender"]=handled_outliers['Gender'].apply(handle_gender)
print("checking email outliers: ",end="")
email_handled_outliers=handled_outliers["Email"].apply(validate_email)
print(handled_outliers[~email_handled_outliers]["Email"].values.tolist(),end="\n\n")

handled_outliers["Email"]=handled_outliers["Email"].apply(email_winsorizing).dropna().reset_index(drop=True)
#handled_outliers=handled_outliers[email_handled_outliers]
handled_outliers

(5, 9)
checking outliers between (-1.5, 1.5) using zscore:
 {'Age': []}
checking GPA column outliers
[4.099999904632568, 5.5, 5.0]
checking email outliers: ['johndoeemail.com', 'johndoeemail.com']



,StudentID,Name,Age,Gender,Email,Phone,City,GPA,Join_Date
0,1,John Doe,25,M,johndoe@email.com,123-456-7890,New York,3.5,2020-01-15
1,2,Jane Smith,23,F,janesmith@email.com,987-654-3210,Los Angeles,4.0,2019-12-10
2,3,Robert Johnson,22,M,johndoe@email.com,555-555-5555,Chicago,4.0,2021-02-18
3,4,Emily Davis,21,F,emily@email.com,111-222-3333,San Francisco,4.0,2020-03-25
4,5,Michael Brown,26,M,michael@email.com,999-888-7777,Houston,3.8,2018-11-05


**7. Categorical Data Standardization**

In [ ]:
handled_data_standardization=handled_outliers.copy()
handled_data_standardization['Gender'] = handled_data_standardization['Gender'].str.upper()
handled_data_standardization['Email'] = handled_data_standardization['Email'].str.strip().str.lower()#
#remaining part handled during outlier detection
handled_data_standardization

,StudentID,Name,Age,Gender,Email,Phone,City,GPA,Join_Date
0,1,John Doe,25,M,johndoe@email.com,123-456-7890,New York,3.5,2020-01-15
1,2,Jane Smith,23,F,janesmith@email.com,987-654-3210,Los Angeles,4.0,2019-12-10
2,3,Robert Johnson,22,M,johndoe@email.com,555-555-5555,Chicago,4.0,2021-02-18
3,4,Emily Davis,21,F,emily@email.com,111-222-3333,San Francisco,4.0,2020-03-25
4,5,Michael Brown,26,M,michael@email.com,999-888-7777,Houston,3.8,2018-11-05


**8. Text Data Cleaning**

In [ ]:
handled_data_cleaning=handled_data_standardization.copy()
handled_data_cleaning=handled_data_cleaning.apply(remove_white_spaces) # remove white spaces
handled_data_cleaning["City"]=handled_data_cleaning["City"].str.capitalize() #  making consistent by first letter capital.
handled_data_cleaning['Phone'] = handled_data_cleaning['Phone'].str.replace(r'[^\d]', '', regex=True)
handled_data_cleaning



,StudentID,Name,Age,Gender,Email,Phone,City,GPA,Join_Date
0,1,John Doe,25,M,johndoe@email.com,1234567890,New york,3.5,2020-01-15
1,2,Jane Smith,23,F,janesmith@email.com,9876543210,Los angeles,4.0,2019-12-10
2,3,Robert Johnson,22,M,johndoe@email.com,5555555555,Chicago,4.0,2021-02-18
3,4,Emily Davis,21,F,emily@email.com,1112223333,San francisco,4.0,2020-03-25
4,5,Michael Brown,26,M,michael@email.com,9998887777,Houston,3.8,2018-11-05


**9. Date Format Standardization**

In [ ]:
handled_data_format=handled_data_cleaning.copy()
handled_data_format["Join_Date"]=pd.to_datetime(handled_data_format["Join_Date"],utc=True,format="%Y-%m-%d", errors='coerce') # time is also converted to utc
handled_data_format


,StudentID,Name,Age,Gender,Email,Phone,City,GPA,Join_Date
0,1,John Doe,25,M,johndoe@email.com,1234567890,New york,3.5,2020-01-15 00:00:00+00:00
1,2,Jane Smith,23,F,janesmith@email.com,9876543210,Los angeles,4.0,2019-12-10 00:00:00+00:00
2,3,Robert Johnson,22,M,johndoe@email.com,5555555555,Chicago,4.0,2021-02-18 00:00:00+00:00
3,4,Emily Davis,21,F,emily@email.com,1112223333,San francisco,4.0,2020-03-25 00:00:00+00:00
4,5,Michael Brown,26,M,michael@email.com,9998887777,Houston,3.8,2018-11-05 00:00:00+00:00


**10. Data Export**

In [ ]:
handled_data_format.to_csv('cleaned_dirty_data.csv', index=False)

**11. Documentation**

In [ ]:
st="Stages"
st_v=[
    "Data Loading",
    "Data Inspection",
    "Handling Missing Values",
    "Handling Duplicate Records",
"Data Type Conversion",
    "Outlier Detection and Handling",
"Categorical Data Standardization",
    "Text Data Cleaning",
    "Date Format Standardization"
    ]
desc="Description"
desc_v=[
    "Data is loaded without index.",
    "The shape of dataset was (5,9), which was very small. It means we have to save rows somehow. Most of the types of the columns were objects. Every columns dtype should be relevant. Only two missing values found on column age and email and there were no duplication as rows were unique. Join_date, Gender and Email values are not consistent and validated, which has be to fixed. Age should be in int. GPA values contains outliers and skewed. Specials characters are in phone column which need to be clean.",
  "we have multiple ways to handle missing values like ffill, bfill, imputing mean values, or remove rows. but in the program, hybrid approach is used like imputing as well as removing based on some condition. Removing condition is  row > int(columns.length/threshold) and the negation of this conditon will use as imputing. In this way, we can avoid values where data is too mouch loss on any row and if row contains only few missing values we can impute value so that we sure that we cannot loss any important information",
    "we donot found any data duplication records. still we made the logic that if we found we should keep the last row alive.",
    "In the stage, we make a method convert_dtype in utility section, this method convert all data types but first it need preset types that was decided manually.",
"At this stage, for numerical values we use zscore and the threshold is (-1.5, 1.5) and remove all the values which are not within this range, but unfortunately all values are between threshold. Next we handle Gender columns and make it Categorical into two unique M for male and F for female. Then we handle gpa which are greater than 4 and emails which are not valid, as dataset is too small removing rows should be last priority. However we have multiple transformation method which handle outliers like Log, winsorizing, Box-cox, we have choose winsorizing in this dataset for both email and GPA.",
    "for categorical values, we update the col Gender values and make is consistent by chosing only 2 Letter M for male and F for female",
    "during data cleaning we remove trailing and leading white spaces, capitalize the data and remove special character if found any.",
    "we use utc based date time and converted into YYYY-MM-DD format"
]
documentation=pd.DataFrame({st:st_v,desc:desc_v})

documentation.to_csv("documentation.csv",index=False)

**12. Summary Report**

In [ ]:
print("data is clean and here is the summary ")
print()
for index, row in documentation.iterrows():
  label=row["Stages"]
  value=row["Description"]
  print(f"{label}: {value}")

data is clean and here is the summary 

Data Loading: Data is loaded without index.
Data Inspection: The shape of dataset was (5,9), which was very small. It means we have to save rows somehow. Most of the types of the columns were objects. Every columns dtype should be relevant. Only two missing values found on column age and email and there were no duplication as rows were unique. Join_date, Gender and Email values are not consistent and validated, which has be to fixed. Age should be in int. GPA values contains outliers and skewed. Specials characters are in phone column which need to be clean.
Handling Missing Values: we have multiple ways to handle missing values like ffill, bfill, imputing mean values, or remove rows. but in the program, hybrid approach is used like imputing as well as removing based on some condition. Removing condition is  row > int(columns.length/threshold) and the negation of this conditon will use as imputing. In this way, we can avoid values where data is t